# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/Users/hussnainaslam/husnain-data/udacity/udacity_cassandra_data_modeling


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:

query = """
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
try:
    session.execute(query)
except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## #
## Query 1:  
### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### For this query, we required 5 columns from file to create table. 3 to select data and 2 for conditions.
#### DataTypes of columns can easliy understand from create statement.
#### To uniquely identify each row primary key consist of session_id, item_in_session. session_id as partitioning key. item_in_session as clustering key

In [8]:
# Create table music_data
query = """
    CREATE TABLE IF NOT EXISTS music_data (
        artist TEXT, 
        song_title TEXT, 
        length FLOAT, 
        session_id INT, 
        item_in_session INT, 
        PRIMARY KEY (session_id, item_in_session)
    )
    """
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Populate Data from csv to the Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_data (artist, song_title, length, session_id, item_in_session) VALUES "
        query = query + "(%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [10]:
# SELECT statement to verify the data was entered into the table
query = """Select artist, song_title, length FROM music_data WHERE session_id = 338 AND item_in_session = 4"""
try:
    result = session.execute(query)
except Exception as e:
    print(e)
    
for row in result:
    print(row[0], row[1], row[2])

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## #

## Query 2: 
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For this query, we required 7 columns from file to create table. 4 to select data, 2 for conditions and for sort order.
#### DataTypes of columns can easliy understand from create statement.
#### To uniquely identify each row primary key consist of user_id, session_id, item_in_session. (user_id, session_id) as partitioning key. item_in_session as clustering key

In [11]:
# Create Table user_session_data
query = """
    CREATE TABLE IF NOT EXISTS user_session_data (
        artist TEXT, 
        song_title TEXT, 
        user_id INT,
        first_name TEXT, 
        last_name TEXT,  
        session_id INT, 
        item_in_session INT, 
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
    """
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
# Populate Data from csv to the Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_session_data (
            artist, song_title, user_id, first_name, last_name, session_id, item_in_session
            ) VALUES """
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[10]), line[1], line[4], int(line[8]), int(line[3])))

In [13]:
# SELECT statement to verify the data was entered into the table
query = """Select artist, song_title, first_name, last_name FROM user_session_data 
            WHERE user_id = 10 AND session_id = 182"""
try:
    result = session.execute(query)
except Exception as e:
    print(e)
    
for row in result:
    print(row[0], row[1], row[2], row[3])

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## #
## Query 3: 
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### For this query, we required 4 columns from file to create table. 2 to select data and 1 for conditions 1 as part of primary key.
#### DataTypes of columns can easliy understand from create statement.
#### To uniquely identify each row primary key consist of song_title, user_id. song_title as partitioning key. user_id as clustering key

In [14]:
# Create Table user_play_date
query = """
    CREATE TABLE IF NOT EXISTS user_play_data (
        song_title TEXT, 
        user_id INT, 
        first_name TEXT, 
        last_name TEXT, 
        PRIMARY KEY (song_title, user_id)
    )
    """
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
# Populate Data from csv to the Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_play_data (song_title, user_id, first_name, last_name) VALUES "
        query = query + "(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [16]:
# SELECT statement to verify the data was entered into the table
query = """Select first_name, last_name FROM user_play_data WHERE song_title = 'All Hands Against His Own'"""
try:
    result = session.execute(query)
except Exception as e:
    print(e)
    
for row in result:
    print(row[0], row[1])



Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS music_data"
try:
    result = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "DROP TABLE IF EXISTS user_session_data"
try:
    result = session.execute(query)
except Exception as e:
    print(e)
    

In [19]:
query = "DROP TABLE IF EXISTS user_play_data"
try:
    result = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()